Network engineers routinely need to verify the routing protocol information in the network. This involves connecting to the various network devices and executing a series of show comamnds. In a multi-vendor network, network engineers must remember the exact command syntax and result output formats making this task much more complex then it should. However, with Batfish and the vendor-independent models that it produces, this task is extremely simple.

In this notebook, we will look at how you can extract routing protocol related information from Batfish.

![Analytics](https://ga-beacon.appspot.com/UA-100596389-3/open-source/pybatfish/jupyter_notebooks/intro-route-analysis?pixel&useReferer)

In [4]:
# Importing packages and loading questions
%run startup.py

In [5]:
NETWORK_NAME = "example_network"
SNAPSHOT_NAME = "example_snapshot"

# please replace this with the full path to the test_rigs folder in your local git repository for Batfish
SNAPSHOT_PATH = "../test_rigs/example"

bf_set_network(NETWORK_NAME)
bf_init_snapshot(SNAPSHOT_PATH, name=SNAPSHOT_NAME, overwrite=True)

'example_snapshot'

The network snapshot that we initialized above is illustrated below. You can download/view devices' configuration files [here](https://github.com/batfish/batfish/tree/master/test_rigs/example).

![example-network](https://raw.githubusercontent.com/batfish/batfish/master/test_rigs/example/example-network.png)

All of the information we will show you in this notebook is dynamically computed by Batfish based on the configuration files for the network devices.

Let's start with listing all of the Layer 3 (IP) neighbors of the router **as2border2**

In [6]:
#insert code to display lanNeighbors for as2border2
ip_neighbors = bfq.neighbors(node1Regex="as2border2", neighborTypes=["lan"]).answer()
ip_neighbors

{'answerElements': [{'class': 'org.batfish.question.NeighborsQuestionPlugin$NeighborsAnswerElement',
   'lanNeighbors': [{'node1': 'as2border2',
     'node1interface': 'GigabitEthernet0/0',
     'node2': 'as3border1',
     'node2interface': 'GigabitEthernet1/0'},
    {'node1': 'as2border2',
     'node1interface': 'GigabitEthernet1/0',
     'node2': 'as2core2',
     'node2interface': 'GigabitEthernet0/0'},
    {'node1': 'as2border2',
     'node1interface': 'GigabitEthernet2/0',
     'node2': 'as2core1',
     'node2interface': 'GigabitEthernet1/0'}]}],
 'question': {'class': 'org.batfish.question.NeighborsQuestionPlugin$NeighborsQuestion',
  'neighborTypes': ['lan'],
  'node1Regex': 'as2border2',
  'node2Regex': '.*',
  'style': 'summary',
  'differential': False,
  'includeOneTableKeys': True,
  'instance': {'description': 'Lists neighbor relationships in the testrig.',
   'instanceName': 'neighbors_f45b5f6d-c036-4623-8a51-61865c9ad592',
   'longDescription': 'Details coming...',
   'ta

Now let's get the list of VRFs configured on **as2border** and the interfaces that are part of each of them

In [7]:
#insert code to diplay VRF list for as2border2 and the interfaces that are in each VRF
interface_vrf = bfq.interfaceProperties(nodeRegex="as2border2", propertySpec="vrf|interface").answer().frame()
interface_vrf

,interface,vrf
0,as2border2:Ethernet0/0,default
1,as2border2:GigabitEthernet0/0,default
2,as2border2:GigabitEthernet1/0,default
3,as2border2:GigabitEthernet2/0,default
4,as2border2:Loopback0,default


In the example network there are no configured VRFs, so all interfaces are part of the **default** VRF. This is sometimes called the **global VRF**.

Let's look at the routing table for the **default** VRF on **as2border**

In [8]:
bf_generate_dataplane()
#insert code to just retrieve the routes for as2border2 default VRF
routing_table = bfq.routes(nodeRegex="as2border2", vrfRegex="default").answer().frame()
routing_table

,Node,VRF,Network,Protocol,Tag,NextHopIp,NextHop,AdminDistance,Metric
0,as2border2,default,1.0.1.0/24,ibgp,-1,10.12.11.1,as1border1,200,50
1,as2border2,default,1.0.1.0/24,ibgp,-1,10.12.11.1,as1border1,200,50
2,as2border2,default,1.0.2.0/24,ibgp,-1,10.12.11.1,as1border1,200,50
3,as2border2,default,1.0.2.0/24,ibgp,-1,10.12.11.1,as1border1,200,50
4,as2border2,default,10.12.11.0/24,ospfE2,-1,2.12.21.2,as2core1,110,20
5,as2border2,default,10.12.11.0/24,ospfE2,-1,2.12.22.2,as2core2,110,20
6,as2border2,default,10.23.21.0/24,connected,-1,AUTO/NONE(-1l),None,0,0
7,as2border2,default,10.23.21.2/32,local,-1,AUTO/NONE(-1l),None,0,0
8,as2border2,default,2.1.1.1/32,ospf,-1,2.12.21.2,as2core1,110,3
9,as2border2,default,2.1.1.1/32,ospf,-1,2.12.22.2,as2core2,110,3


Often times, network engineers will check the routing tables for multiple devices in order to chase down a connectivity issue.

So now, let's find all of the devices that have a route for **1.0.2.0/24**

In [9]:
#insert code to just retrieve routing entries for target prefix on ALL devices in the snapshot

Figuring out which routers have a given prefix installed is useful, but we also want to know which routers do not have this prefix.
So let's give that a try now.

In [10]:
#insert code to just return the list of devices that DO NOT have the target prefix in the routing table

That's it for this notebook. In this notebook we should you:

a) How to retrieve the list of Layer 3 neighbors for any router in the network
b) Retrieve the list of VRFs, and the interfaces in each VRF for any router
c) How to find all routers that have a specific prefix in the routing table
d) How to find all routers that are misisng a specific prefix in the routing table

These are foundational concepts for using Batfish to perform route analysis. In future notebooks we will dive into more advanced route analysis using Batfish.

### Want to know more? 

Reach out to us through [Slack](https://join.slack.com/t/batfish-org/shared_invite/enQtMzA0Nzg2OTAzNzQ1LTUxOTJlY2YyNTVlNGQ3MTJkOTIwZTU2YjY3YzRjZWFiYzE4ODE5ODZiNjA4NGI5NTJhZmU2ZTllOTMwZDhjMzA) or [Github](https://github.com/batfish/batfish) to learn more. We would love to hear about your network and talk about how Batfish can help improve it's security and reliability.

We hope that you will continue to explore **Batfish** and all that it has to offer.